# 01 Basic Analysis with GeoAnomalyMapper (GAM)

This notebook provides a step-by-step introduction to using GAM for subsurface anomaly detection. We'll analyze the Giza Pyramids region using gravity and seismic data.

**Objectives**:
- Import GAM and set up the environment.
- Load sample configuration.
- Run a basic pipeline.
- Visualize and interpret results.
- Export outputs.

**Prerequisites**: GAM installed with `[geophysics,visualization]` extras. Run in Jupyter with Python 3.10+.

In [ ]:
# Install if needed (run once)
# !pip install geoanomalymapper[geophysics,visualization] jupyter

# Imports
import yaml
import pandas as pd
import matplotlib.pyplot as plt
from gam.core.pipeline import run_pipeline
from gam.visualization.manager import generate_visualization

# Verify GAM
import gam
print(f"GAM version: {gam.__version__}")

## Step 1: Set Up Environment and Configuration

Define the bounding box for Giza (small region for quick run) and load config.

In [ ]:
# Giza bbox: approx Great Pyramid area
bbox = (29.95, 30.05, 31.10, 31.20)  # min_lat, max_lat, min_lon, max_lon
modalities = ["gravity", "seismic"]  # Start simple

# Basic config (expand as needed)
config = {
    "data": {
        "bbox": bbox,
        "modalities": modalities,
        "cache_dir": "../data/cache"
    },
    "preprocessing": {
        "grid_res": 0.01  # Fine grid for local
    },
    "modeling": {
        "threshold": 2.0
    },
    "visualization": {
        "map_type": "2d",
        "export_formats": ["png", "csv"]
    },
    "core": {
        "output_dir": "../results/giza_basic"
    }
}

# Save config for reuse
with open("giza_basic_config.yaml", "w") as f:
    yaml.dump(config, f)
print("Config saved to giza_basic_config.yaml")

## Step 2: Run the Pipeline

Execute ingestion, preprocessing, modeling, and detection. This fetches public data and processes it.

In [ ]:
# Run pipeline (may take 5-10 min first time due to data fetch)
results = run_pipeline(
    bbox=bbox,
    modalities=modalities,
    config=config,
    output_dir=config["core"]["output_dir"]
)

# Inspect results
print("Pipeline completed!")
print(f"Detected {len(results['anomalies'])} anomalies")
print(results['anomalies'].head())

## Step 3: Visualize Results

Generate a 2D map of anomalies. High-confidence spots may indicate subsurface features.

In [ ]:
# Generate visualization
anomalies = results['anomalies']
viz_paths = generate_visualization(
    anomalies,
    type="2d",
    color_scheme="viridis",
    output_dir=config["core"]["output_dir"]
)

# Display map (if PNG)
import matplotlib.image as mpimg
img = mpimg.imread(viz_paths['png'])
plt.figure(figsize=(10, 8))
plt.imshow(img)
plt.axis('off')
plt.title("Giza Anomaly Map (Viridis Colormap)")
plt.show()

print("Visualization saved to:", viz_paths)

## Step 4: Interpret Outputs

Examine anomalies: Focus on high-confidence (>0.7) near known sites (e.g., Great Pyramid at ~29.979 N, 31.134 E).

In [ ]:
# Filter high-confidence anomalies
high_conf = anomalies[anomalies['confidence'] > 0.7]
print("High-confidence anomalies:")
print(high_conf[['lat', 'lon', 'depth', 'confidence', 'anomaly_type']])

# Plot locations
plt.figure(figsize=(8, 6))
plt.scatter(high_conf['lon'], high_conf['lat'], c=high_conf['confidence'], s=50, cmap='Reds')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('High-Confidence Anomalies in Giza')
plt.colorbar(label='Confidence')
plt.grid(True, alpha=0.3)
plt.show()

# Interpretation: Depths < -100m may indicate voids; cross-check with literature.

## Step 5: Export Results

Save for further analysis in GIS or reports.

In [ ]:
# Export anomalies to CSV (already in results, but explicit)
anomalies.to_csv(config["core"]["output_dir"] + "/anomalies_detailed.csv", index=False)

# Export model (if needed)
if 'fused_model' in results:
    results['fused_model'].to_netcdf(config["core"]["output_dir"] + "/fused_model.nc")

print("Exports complete. Check output_dir for files.")

# Next: Try adding 'insar' modality or larger bbox in 02_multi_modality_fusion.ipynb